In [1]:
import cobra
from cobra.io.mat import *
import warnings
warnings.simplefilter('ignore')
#from cobra.sampling import sample
from cobra.sampling import OptGPSampler, ACHRSampler
import os
cpus = os.cpu_count()
from cobra.io.json import load_json_model
#model = load_matlab_model("./COBRA_models/GEM_Recon2_thermocurated_redHUMAN.mat")
model = load_matlab_model("./COBRA_models/GEM_Recon3_thermocurated_redHUMAN_AA.mat")

recon301 =load_matlab_model("./COBRA_models/Recon3DModel_301.mat")


No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, i, l, m, n, r, x
No defined compartments in model Recon3DModel. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, i, l, m, n, r, x


In [5]:

print(
model.optimize(),recon301.optimize(),
cpus)

len([ r.id for r in model.reactions])- len([ r.id for r in recon301.reactions])

<Solution 31.104 at 0x7efc35ecb070> <Solution 755.003 at 0x7efc35ecacb0> 20


2

In [10]:


model.reactions.get_by_id("r0403").bounds 
model.reactions.get_by_id("r0402").bounds 
model.reactions.get_by_id("TRPHYDRO2").bounds 
model.reactions.get_by_id("THBPT4ACAMDASE").bounds 


model.reactions.get_by_id("RE1709C").bounds 

(-10, 10)

In [9]:
import re
from itertools import compress



rxns        = [ r.id for r in model.reactions]
subsystems  = [s.subsystem for s in model.reactions]
subsys_bool = [bool(re.search("Phen|phen|Tetrahydrobiopterin", s)) for s in subsystems]
subsys_rxns = list(compress(rxns, subsys_bool))

reversible = [
'PHETA1',
'PHETA1m',
'r0547',
'RE0830C',
'RE0830N',
'RE1709C',
'RE1709N',
'RE2660C',
'RE2660N']

forward_rxns =  list(set(subsys_rxns)-set(reversible))

for r in forward_rxns:
    model.reactions.get_by_id(r).bounds  = (1e-8, 10)
    
    
for r in reversible:
    model.reactions.get_by_id(r).bounds  = (-10, 10)

co    = model.problem.Constraint(model.reactions.biomass.flux_expression,
                                 lb=.90 * model.optimize().objective_value,
                                 ub=.999 * model.optimize().objective_value)
model.add_cons_vars([co])
assert model.optimize().objective_value > 0
print(
model.optimize(),recon301.optimize(),
cpus)
forward_rxns

<Solution 31.042 at 0x7efc350580a0> <Solution 755.003 at 0x7efc350583d0> 20


['PACCOALm',
 '3HPPPNOHGLUCc',
 'HMR_7628',
 'HMR_6834',
 'DHPR',
 'HMR_6784',
 'HMR_6782',
 'r0545',
 'HMR_6729',
 'PACCOAL',
 'r0403',
 '3HPPPNOHc',
 'HMR_4816',
 'PHLAC',
 'PTHPS',
 'PEAMNO',
 'HMR_6728',
 'HMR_6786',
 'HMR_6876',
 'PHETHPTOX2',
 'PHYCBOXL',
 'HMR_6844',
 'HMR_6850',
 'HMR_6854',
 'SPR',
 'r0398',
 'HMR_6790',
 'HMR_6839',
 'HMR_6770',
 'GLYNATm',
 'HMR_7756',
 'THBPT4ACAMDASE',
 'HMR_6785',
 'r0673',
 'HMR_6826',
 'HMR_6838',
 'HMR_6747',
 'PPOR',
 'HMR_6755',
 'r0402',
 'PHACCOAGLNAC',
 'GTPCI',
 'HMR_6874',
 'DHPR2',
 'r0399']

In [3]:
model.reactions.get_by_id("r0399").bounds       = (0, 1e-3)
model.reactions.get_by_id("PHETHPTOX2").bounds  = (0, 1e-3)


assert model.optimize().objective_value > 0
model.optimize().objective_value, model.optimize().to_frame().loc["r0399"], model.optimize().to_frame().loc["PHETHPTOX2"]




(30.793010052375,
 fluxes           0.0
 reduced_costs    0.0
 Name: r0399, dtype: float64,
 fluxes           0.0
 reduced_costs    0.0
 Name: PHETHPTOX2, dtype: float64)

In [4]:
optgp = OptGPSampler(model, processes=cpus, thinning=100)
samples = optgp.sample(7_000)
samples.to_parquet("./results/fluxes/flux_samples_PKU_7_000.parquet.gzip", compression='gzip')  

import pandas as pd
flux_samples_PKU_7_000 = pd.read_parquet("./results/fluxes/flux_samples_PKU_7_000.parquet.gzip")  
flux_samples_PKU_7_000[["PHETHPTOX2", "r0399"]]

,PHETHPTOX2,r0399
0,0.000011,0.000011
1,0.000011,0.000011
2,0.000011,0.000011
3,0.000011,0.000011
4,0.000011,0.000011
...,...,...
6995,0.000480,0.000022
6996,0.000404,0.000065
6997,0.000391,0.000065
6998,0.000388,0.000062


In [5]:
model.reactions.get_by_id("r0399").bounds       = (30, 100)
model.reactions.get_by_id("PHETHPTOX2").bounds  = (30, 100)
assert model.optimize().objective_value > 0
model.optimize().to_frame().loc["r0399"], model.optimize().to_frame().loc["PHETHPTOX2"]
model.optimize()

,fluxes,reduced_costs
10FTHF5GLUtl,0.000000,0.0
10FTHF5GLUtm,0.000000,0.0
10FTHF6GLUtl,0.000000,0.0
10FTHF6GLUtm,0.000000,0.0
10FTHF7GLUtl,0.000000,0.0
...,...,...
NADH2_u10mi,0.000000,0.0
CYOOm3i,0.000000,0.0
CYOOm2i,41.666667,0.0
ARTPLM1,0.000000,0.0


In [6]:

optgp = OptGPSampler(model, processes=cpus, thinning=100)
samples = optgp.sample(7_000)
samples.to_parquet("./results/fluxes/flux_samples_CONTROL_7_000.parquet.gzip", compression='gzip')  
import pandas as pd
flux_samples_CONTROL_7_000 = pd.read_parquet("./results/fluxes/flux_samples_CONTROL_7_000.parquet.gzip")  
flux_samples_CONTROL_7_000[["PHETHPTOX2", "r0399"]]

,PHETHPTOX2,r0399
0,30.0,30.038785
1,30.0,30.039390
2,30.0,30.037066
3,30.0,30.039025
4,30.0,30.037152
...,...,...
6995,30.0,40.786187
6996,30.0,40.774335
6997,30.0,40.745141
6998,30.0,40.782505


optgp = OptGPSampler(model, processes=cpus, thinning=1000)
samples = optgp.sample(10_000)
samples.to_parquet("./results/fluxes/flux_samples.parquet.gzip", compression='gzip')  

### Get non zero samples from phenylalanine and biopterin subsystems

In [1]:
import pandas as pd
from cobra.io.mat import *
from itertools import compress

model                      = load_matlab_model("./COBRA_models/GEM_Recon3_thermocurated_redHUMAN_AA.mat")
flux_samples_CONTROL_7_000 = pd.read_parquet("./results/fluxes/flux_samples_CONTROL_7_000.parquet.gzip")  
flux_samples_PKU_7_000     = pd.read_parquet("./results/fluxes/flux_samples_PKU_7_000.parquet.gzip")  


No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, i, l, m, n, r, x


In [3]:


rxns        = [ r.id for r in model.reactions]
subsystems  = [s.subsystem for s in model.reactions]
subsys_bool = [bool(re.search("Phen|phen|Tetrahydrobiopterin", s)) for s in subsystems]
pd.Series(subsystems).iloc[subsys_bool].unique()


array(['Tetrahydrobiopterin metabolism', 'Phenylalanine metabolism'],
      dtype=object)

In [4]:
subsys_rxns = list(compress(rxns, subsys_bool))
subsys_rxns

['DHPR2',
 'PACCOAL',
 'PEAMNO',
 'PHACCOAGLNAC',
 'PHETA1m',
 'PHETHPTOX2',
 'PHYCBOXL',
 'PPOR',
 'PTHPS',
 'SPR',
 'THBPT4ACAMDASE',
 'r0399',
 'r0402',
 'r0403',
 'r0545',
 'r0547',
 'r0673',
 'RE0830C',
 'RE0830N',
 'RE1709C',
 'RE1709N',
 'RE2660C',
 'RE2660N',
 'PHLAC',
 '3HPPPNOHc',
 '3HPPPNOHGLUCc',
 'GLYNATm',
 'PACCOALm',
 'DHPR',
 'GTPCI',
 'r0398',
 'PHETA1',
 'HMR_6728',
 'HMR_6729',
 'HMR_6747',
 'HMR_6755',
 'HMR_6770',
 'HMR_6782',
 'HMR_6784',
 'HMR_6785',
 'HMR_6786',
 'HMR_6790',
 'HMR_6826',
 'HMR_6834',
 'HMR_6838',
 'HMR_6839',
 'HMR_6844',
 'HMR_6850',
 'HMR_6854',
 'HMR_6874',
 'HMR_6876',
 'HMR_7628',
 'HMR_7756',
 'HMR_4816']

In [13]:

def samples_subsys_rxns(samples, subsys_rxns):

    samples_subsys_rxns = samples[subsys_rxns]

    return samples_subsys_rxns.loc[:,
            list(samples_subsys_rxns.sum().abs() > 1e-10 )]


samples_subsys_CONTROL_7_000 = samples_subsys_rxns(flux_samples_CONTROL_7_000, subsys_rxns)
samples_subsys_PKU_7_000 = samples_subsys_rxns(flux_samples_PKU_7_000, subsys_rxns)
all(samples_subsys_CONTROL_7_000.columns == samples_subsys_PKU_7_000.columns)

#samples_subsys_rxns.loc[:,
#list(samples_subsys_rxns.sum().abs() > 1e-7 )].to_parquet("./results/fluxes/subsys_fluxes_non_zero.parquet.gzip")


True

In [15]:
samples_subsys_CONTROL_7_000.loc[:,
list(samples_subsys_CONTROL_7_000.sum().abs() > 1e-7 )].to_parquet("./results/fluxes/samples_subsys_CONTROL_7_000.parquet.gzip")
samples_subsys_PKU_7_000.loc[:,
list(samples_subsys_PKU_7_000.sum().abs() > 1e-7 )].to_parquet("./results/fluxes/samples_subsys_PKU_7_000.parquet.gzip")